# Offline augmentation

Offline augmentation is a standard augmentation technique, where the training set is augmented before the training procedure. It has been proved that dataset augmentation improves generalization capabilities of a model.

More detailed description of the offline augmentation can be found in our [paper](https://arxiv.org/abs/1903.05580).

In [ ]:

import os
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from python_research.keras_models import build_1d_model
from python_research.dataset_structures import HyperspectralDataset, BalancedSubset
from python_research.augmentation.transformations import PCATransformation, \
    StdDevNoiseTransformation
from python_research.augmentation.offline_augmenter import OfflineAugmenter

In [ ]:
DATA_DIR = os.path.join('..', '..', 'hypernet-data')
RESULTS_DIR = os.path.join('..', '..', 'hypernet-data', 'results', 'offline_augmentation')
DATASET_PATH = os.path.join(DATA_DIR, '')
GT_PATH = os.path.join(DATA_DIR, '')
os.makedirs(RESULTS_DIR, exist_ok=True)

# Prepare the data

Extract the training, validation and test sets. Trainig set will be balanced (each class will have equal number of samples)

In [ ]:
# Number of samples to be extracted from each class as training samples
SAMPLES_PER_CLASS = 300 
# Percentage of the training set to be extracted as validation set 
VAL_PART = 0.1

test_data = HyperspectralDataset(DATASET_PATH, GT_PATH)

test_data.normalize_labels()
test_data.expand_dims(axis=-1)

# Extract training and validation sets
train_data = BalancedSubset(test_data, SAMPLES_PER_CLASS)
val_data = BalancedSubset(train_data, VAL_PART)

# Data normalization

Data is normalized using Min-Max feature scaling. Min and max values are extracted from train and test sets.

In [ ]:
max_ = train_data.max if train_data.max > val_data.max else val_data.max
min_ = train_data.min if train_data.min < val_data.min else val_data.min
train_data.normalize_min_max(min_=min_, max_=max_)
val_data.normalize_min_max(min_=min_, max_=max_)
test_data.normalize_min_max(min_=min_, max_=max_)

# Build the model

In [ ]:
# Number of epochs without improvement on validation set after which the 
# training will be terminated 
PATIENCE = 15 
# Number of kernels in the first convolutional layer
KERNELS = 200 
# Size of the kernel in the first convolutional layer
KERNEL_SIZE = 5 
# Number of classes in the dataset
CLASSES_COUNT = 16 

# Build 1d model
model = build_1d_model((test_data.shape[1:]), KERNELS,
                       KERNEL_SIZE, CLASSES_COUNT)
print(model.summary())


# Augmentation

There are four different types of augmentation implemented:
* __Noise injection__ - For band of a given pixel, a random value from normal distribution with mean = 0 and standard deviation equal to the standard deviation of pixel's class and particular band is drawn, multiplied by scaling factor (a = 0.25) and added to the original value
* __PCA-based augmentation__ - Method based on PCA. In the first step, principal components are calculated on a training set. Then, a pixel under consideration is transformed using previously calculated principal components, first value of the resulting vector is multiplied by a random value from a given range (0.9 - 1.1 on default), and an inverse transformation is performed on such a vector, resulting in an augmented sample.
* __Highlighting/dimming__ - To each band of a given sample, a percentage (10% on default) of an average value of that band (across all samples in the training set) is added (highlighting) or subtracted (dimming)
* __Generative Adversarial Network (GAN)__ - covered in a separate jupyter notebok

The augmentation is performed using **`OfflineAugmenter`** class, accepting an objects of type **`Transformation`**, which encapsulates the augmentation logic. The **`Transformation`** objects need to call the `fit` method before using it for augmentation, in order to collect all necessary information about the set.

The `sampling_mode` argument indicates how many samples will be augmented. If `twice`, the number of samples in each class will be doubled. If `max_twice`, number of samples in each class will be doubled, unless the doubled count exceeds the samples count in the most numerous class, in which case the number of augmented samples will be equal to the difference between those two classes' samples count.

In [ ]:
# Remove last dimension (convert column vectors to row vectors)
train_data.data = train_data.get_data()[:, :, 0]

# Augment training set using PCA
pca_transformation = PCATransformation(n_components=train_data.shape[-1])
pca_transformation.fit(train_data.get_data())
offline_augmenter = OfflineAugmenter(pca_transformation, sampling_mode='twice')
augmented_data, augmented_labels = offline_augmenter.augment(train_data)

# Add augmented samples to the training set
train_data.vstack(augmented_data)
train_data.hstack(augmented_labels)

train_data.expand_dims(axis=-1)

checkpoint = ModelCheckpoint(os.path.join(RESULTS_DIR, 
                                          "offline_augmentation_pca_augmented") + 
                                          "_model",
                             save_best_only=True)
early = EarlyStopping(patience=PATIENCE)
BATCH_SIZE = 64 
EPOCHS = 200 
# Train model on augmented dataset
history = model.fit(x=train_data.get_data(),
                    y=train_data.get_one_hot_labels(CLASSES_COUNT),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=2,
                    callbacks=[early, checkpoint],
                    validation_data=(val_data.get_data(),
                                     val_data.get_one_hot_labels(CLASSES_COUNT)))
# Load best model
model = load_model(os.path.join(RESULTS_DIR, 
                                "offline_augmentation_pca_augmented") + 
                                "_model")

# Calculate test set score with PCA-based augmentation
test_score = model.evaluate(x=test_data.get_data(),
                            y=test_data.get_one_hot_labels(CLASSES_COUNT))
print("Test set score with PCA offline augmentation: {}".format(test_score[1]))


In [ ]:
# Remove last dimension (convert column vectors to row vectors)
train_data.data = train_data.get_data()[:, :, 0]

# Augment training set using noise injection
noise_transformation = StdDevNoiseTransformation(alphas=[0.1])
noise_transformation.fit(train_data.get_data())
offline_augmenter = OfflineAugmenter(noise_transformation, 
                                     sampling_mode='twice')
augmented_data, augmented_labels = offline_augmenter.augment(train_data)

# Add augmented samples to the training set
train_data.vstack(augmented_data)
train_data.hstack(augmented_labels)

# Convert row vectors to columns vectors (keras requirement)
train_data.expand_dims(axis=-1)

checkpoint = ModelCheckpoint(os.path.join(RESULTS_DIR, 
                                          "offline_augmentation_noise_augmented") + 
                                          "_model",
                             save_best_only=True)
early = EarlyStopping(patience=PATIENCE)
BATCH_SIZE = 64 
EPOCHS = 200
# Train model on augmented dataset
history = model.fit(x=train_data.get_data(),
                    y=train_data.get_one_hot_labels(CLASSES_COUNT),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=2,
                    callbacks=[early, checkpoint],
                    validation_data=(val_data.get_data(),
                                     val_data.get_one_hot_labels(CLASSES_COUNT)))
# Load best model
model = load_model(os.path.join(RESULTS_DIR, 
                                "offline_augmentation_noise_augmented") + 
                                "_model")

# Calculate test set score without augmentation
test_score = model.evaluate(x=test_data.get_data(),
                            y=test_data.get_one_hot_labels(CLASSES_COUNT))
print("Test set score with noise injection offline augmentation: {}".format(test_score[1]))